Funkcja definiująca macierz jednostokową o rozmiarze n

In [1]:
function eye(n)
    return Matrix{Float64}(I,n,n)
end

eye (generic function with 1 method)

Funkcja zwracająca indeksy (wspólrzędne indeksowe) maksymalnych co do modułu elementów ponad diagonalnych.

In [2]:
function maxst(A)
    s = 1;
    t = 2;
    n = size(A,1);
    
    for c = 2:n
        for r = 1:c-1
            if abs(A[r,c]) > abs(A[s,t])
                s = r;
                t = c;
            end
        end
    end
    return s,t
end

maxst (generic function with 1 method)

Funckja sprawdzajaca czy wszystkie elementy pozadiagonalne są równe 0 (mniejsze niz 1e-10)

In [3]:
function isNonDiagZeros(A::Matrix)::Bool
    n = size(A,1);
    isZero = true;
    for i = 1:n
        !isZero && break;
        for j = 1:n
            if i != j && abs(A[i,j]) > 1e-10
                isZero = false;
                break;
            end
        end
    end
    return isZero
end


isNonDiagZeros (generic function with 1 method)

In [4]:
function reverseMatrix(A::Matrix)
    n = size(A,1);
    B=eye(n);
    for i = 1:n
        for j = 1:n
            B[i,n-j+1] = A[i,j]
        end
    end
    return B
end


reverseMatrix (generic function with 1 method)

In [5]:
function jacobi_eigen(A)
    n = size(A,1);  
    Q = eye(n);
    
    while true
        # znajdujemy index s,t najwiekszej wartości nad diagonalą
        s,t = maxst(A);
        d = sqrt((A[s,s] - A[t,t])^2 + 4*A[s,t]^2);
        sin2t = 2*A[s,t]/d;
        cos2t = (A[s,s] - A[t,t]) / d;
        dt = sqrt(2*(1+cos2t));
        sint = abs(sin2t) / dt;
        cost = sign(A[s,t]) * abs((1+ cos2t) / dt);
        
        R = eye(n);
        R[s,s] = cost;
        R[t,t] = cost;
        R[s,t] = -sint;
        R[t,s] = sint;
        A = R'*A*R;
        
        Q = Q*R;
        
        
        #sprawdzamy zbierzność
        if isNonDiagZeros(A)
           break; 
        end
    end
    
    # obcinamy prawie zera, żeby było lepiej widać
    A[abs.(A) .< 1e-10] .= 0.0;
    # bierzemy tylko elementy poza diagonalne i odwracamy
    jacobi_eigen_v = reverse(A[A .!= 0]);
    
    return jacobi_eigen_v, reverseMatrix(Q)
end

jacobi_eigen (generic function with 1 method)

In [6]:
using LinearAlgebra

A = [12 6 -6 1
    6 16 2 100
    -6 2 16 1
    1 100 1 4];

jacobi_eigen_vals, jacobi_eigen_vector = jacobi_eigen(A);
e,v = eigen(A);

println("Wartości własne z jacobi_eigen: ", jacobi_eigen_vals)
println("Wartośći własne z eigen: ", e)

# println("Wektory własne z jacobi_eigen: ", jacobi_eigen_vector)
# show(jacobi_eigen_vector)
# println("Wektory własne z eigen: ", v)


@time jacobi_eigen(A);
@time eigen(A);
display(jacobi_eigen_vector)


Wartości własne z jacobi_eigen: 

4×4 Matrix{Float64}:
  0.033589     0.810494    0.582634     -0.0500884
 -0.685874    -0.0159906  -0.000760996  -0.727544
  0.00796229   0.582556   -0.812518     -0.0194603
  0.726901    -0.058921   -0.0187406    -0.683953

[-90.29875173791679, 7.49632641100462, 20.327366093563793, 110.47505923335348]
Wartośći własne z eigen: [-90.2987517379167, 7.496326411004787, 20.32736609356303, 110.47505923334917]
  0.000044 seconds (68 allocations: 12.562 KiB)
  0.000053 seconds (11 allocations: 2.578 KiB)


Standaryzacja danych

In [125]:
using Statistics

# standarization formula
function standarize_data(A::Matrix)::Matrix{Float64}
    n, m = size(A);
    X = [1:n;];
    
    if typeof(A)==Matrix{Int}
        A = convert(Matrix{Float64}, A); 
    else 
        X = convert(Array{Float64}, X); 
    end
    
    for i=1:m
        for j=1:n
            X[j] = A[j,i];
         end
        m = real(mean(X));
        dev = real(std(X));
        
        for j=1:n
            A[j,i]=(A[j,i]-m)/dev;
        end
    end
    
    return A
end

standarize_data (generic function with 1 method)

Calculate the covariance matrix for the whole dataset

In [126]:
function calcCov(A::Matrix, w::Int64, m::Int64)
    n = size(A, 1);
    val = 1.0;

    for i=1:n
        val = val + (A[i, w]*A[i,m])
    end
    return val/(n-1)
end

function covariance(A::Matrix)
    n, m = size(A);
    s = n;
    
    if n > m
        s = m;
    end
    
    B = zeros(Float64, (s, s))
    
    for i=1:s
        for j=1:s
            B[i,j] = calcCov(A, i, j)
        end
    end
    
    return B
end

covariance (generic function with 1 method)

In [168]:
M= rand(Int, (10,10));

In [174]:
# M = [1 2 3 4;
#      5 5 6 7;
#      1 4 2 3;
#      5 3 2 1;
#      8 1 2 2];

M = [7 4 3;
     4 1 8;
     6 3 5;
     8 6 1;
     8 5 7;
     7 2 9;
     5 3 3;
     9 5 8;
     7 4 5;
     8 2 2];


function customPCA(A::Matrix, tol::Float64)
    
    cov_matrix = covariance(A);        
    val, vec  = jacobi_eigen(cov_matrix);

    # descending order
    desc_val = val[end:-1:1];
    desc_vec = vec[:,end:-1:1];
    
    # percentage of eigenvalues
    weight,index=0,0
    for i in 1:length(desc_val)
        weight+=desc_vec[i];
        if weight/sum(desc_vec)>=tol
            index=i
            break
        end
    end
    
    # Return eigenvector matrix
    return desc_vec[:,1:index];
end

standarized_data = standarize_data(M);
model = customPCA(standarized_data, 0.1);

display(model)
PCA_model = standarized_data*model;
display(PCA_model)

@time customPCA(standarized_data, 0.1);

3×1 Matrix{Float64}:
  0.6904283082973928
  0.7079968110879636
 -0.1484899545116585

10×1 Matrix{Float64}:
  0.38029550982366567
 -2.586772437237256
 -0.6263620779876281
  1.8347295974697433
  1.069524199397203
 -0.8326863880836588
 -0.9736239004723003
  1.4696908382548344
  0.2744858770777468
 -0.009281218242351863

  0.000014 seconds (43 allocations: 5.922 KiB)
